# import data

In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta, time
stk_code = '603355'
exchange = 'sh_exchange'
folder_path = f'/Users/bokie/code/python/CMS/TWAP优化/trading-optimize/stock_data/{exchange}/{stk_code}'

files_pkl = os.listdir(folder_path)
files_pkl.remove('.DS_Store')
files_pkl
stk_dic = {}
for file in files_pkl:
    tmp_path = folder_path + '/' + file
    df_tmp = pd.read_pickle(tmp_path)
    tmp_key = file[7:15]
    stk_dic[tmp_key] = df_tmp

# date = '20220602'
# date = '20220701'
# date = '20220801'
# date = '20220812'
# date = '20220815'
# date = '20220816'
# date = '20220817'
# date = '20220818'
date = '20220819'
date = '20220822'
short_open_threshold = 0.973773 #做空开仓
short_close_threshold = 0 #做空平仓

long_close_threshold = -0 #做多平仓
long_open_threshold = -1.407495 #做多开仓

# start_date = '20220819'
# end_date = '20220819'
start_date = date
end_date = date
start_date_formatted = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
end_date_formatted = end_date[:4] + '-' + end_date[4:6] + '-' + end_date[6:]
# pd.set_option('display.max_rows', None)
# df_stk_orgin['index_col'] = df_stk_orgin['date_time']
# row.set_index('index_col').iloc[:,113:114]    
df_stk_orgin = stk_dic[date]

In [2]:
date[:4] + '-' + date[4:6] + '-' + date[6:]

'2022-08-22'

In [3]:
full_cols = df_stk_orgin.columns.tolist()
# print(full_cols)

In [4]:
# stk_clean_dic = {}

# for k,v in stk_dic.items():
#     v['tmp_index'] = v['date_time'].apply(lambda x:pd.to_datetime(str(x)))
#     k_formatted = k[:4] + '-' + k[4:6] + '-' + k[6:]
#     stk_clean_dic[k] = v.set_index('tmp_index').loc[f'{k_formatted} 09:30:00':f'{k_formatted} 14:55:00']
    

In [5]:
# # stk_clean_dic['20220825']
# df_stk_orgin = pd.DataFrame([])
# tmp_keys = list(stk_clean_dic.keys())
# tmp_keys.sort()
# sorted_keys = tmp_keys[-10:]
# for k in sorted_keys:
#     df_stk_orgin = df_stk_orgin.append(stk_clean_dic[k])

# start_date = sorted_keys[0]
# end_date = sorted_keys[-1]
# start_date_formatted = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
# end_date_formatted = end_date[:4] + '-' + end_date[4:6] + '-' + end_date[6:]

# len(df_stk_orgin)

In [6]:
# print(tmp_keys),print(sorted_keys)

# signal calculate

In [7]:
pd.set_option('display.max_rows', None)
cols = ['date_time', 
'bid_price1','offer_price1',
'bid_price2','offer_price2',
'bid_price3','offer_price3',
'bid_price4','offer_price4',
'bid_price5','offer_price5',
'bid_order_qty1','offer_order_qty1',
'bid_order_qty2','offer_order_qty2',
'bid_order_qty3','offer_order_qty3',
'bid_order_qty4','offer_order_qty4',
'bid_order_qty5','offer_order_qty5',
'last_px']
# df_stk_orgin[cols].head(100)

In [8]:
import numpy as np
# from sqlalchemy import false

df_signal = pd.DataFrame(data=[],columns=['wt','vt'])

pre_bid = np.average(df_stk_orgin['bid_price1'].tolist()[0]+df_stk_orgin['bid_price2'].tolist()[0]+\
                    df_stk_orgin['bid_price3'].tolist()[0]+df_stk_orgin['bid_price4'].tolist()[0]+\
                    df_stk_orgin['bid_price5'].tolist()[0])

pre_offer = np.average(df_stk_orgin['offer_price1'].tolist()[0]+df_stk_orgin['offer_price2'].tolist()[0]+\
                    df_stk_orgin['offer_price3'].tolist()[0]+df_stk_orgin['offer_price4'].tolist()[0]+\
                    df_stk_orgin['offer_price5'].tolist()[0])

pre_bid_qty = df_stk_orgin['bid_order_qty1'].tolist()[0] + df_stk_orgin['bid_order_qty2'].tolist()[0]+\
                df_stk_orgin['bid_order_qty3'].tolist()[0]+df_stk_orgin['bid_order_qty4'].tolist()[0]+\
                df_stk_orgin['bid_order_qty5'].tolist()[0]

pre_offer_qty = df_stk_orgin['offer_order_qty1'].tolist()[0] + df_stk_orgin['offer_order_qty2'].tolist()[0]+\
                df_stk_orgin['offer_order_qty3'].tolist()[0] + df_stk_orgin['offer_order_qty4'].tolist()[0]+\
                df_stk_orgin['offer_order_qty5'].tolist()[0]

for index,row in df_stk_orgin[cols].set_index('date_time').iterrows():
    bid_qty = row['bid_order_qty1']+row['bid_order_qty2']+row['bid_order_qty3']+row['bid_order_qty4']+row['bid_order_qty5']

    offer_qty = row['offer_order_qty1']+row['offer_order_qty2']+row['offer_order_qty3']+row['offer_order_qty4']+row['offer_order_qty5']

    bid_price = np.average(row['bid_price1']+row['bid_price2']+row['bid_price3']+row['bid_price4']+row['bid_price5'])

    offer_price = np.average(row['offer_price1']+row['offer_price2']+row['offer_price3']+row['offer_price4']+row['offer_price5'])

    if bid_price > pre_bid:
        tmp_wt = np.log(bid_qty)

    # assert False
    elif bid_price == pre_bid:
        tmp_wt = np.log(bid_qty) - np.log(pre_bid_qty)
    elif bid_price < pre_bid:
        tmp_wt = -np.log(pre_bid_qty)
    
    if offer_price > pre_offer:
        tmp_vt = -np.log(pre_offer_qty)
    elif offer_price == pre_offer:
        tmp_vt = np.log(offer_qty) - np.log(pre_offer_qty)
    elif offer_price < pre_offer:
        tmp_vt = np.log(offer_qty)

    df_signal.loc[pd.to_datetime(str(index)),'wt'] = tmp_wt #原始的时间标签是int类型，先转成str再转datetime
    df_signal.loc[pd.to_datetime(str(index)),'vt'] = tmp_vt
    df_signal.loc[pd.to_datetime(str(index)),'price'] = row['last_px']
    pre_bid = bid_price
    pre_offer = offer_price
    pre_bid_qty = bid_qty
    pre_offer_qty = offer_qty
df_signal['log-OFI'] = df_signal['wt'] - df_signal['vt']
df_signal['log-OFI-MA50'] = df_signal['log-OFI'].rolling(50).mean()
df_signal['log-OFI-MA100'] = df_signal['log-OFI'].rolling(100).mean()
df_signal['log-OFI-MA200'] = df_signal['log-OFI'].rolling(200).mean()

/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_89330/201006691.py:45: RuntimeWarning: divide by zero encountered in log
  tmp_vt = np.log(offer_qty)
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_89330/201006691.py:41: RuntimeWarning: divide by zero encountered in log
  tmp_vt = -np.log(pre_offer_qty)


In [9]:
# df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()

In [10]:
incicator_stat = df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].dropna().describe()
# incicator_stat
incicator_stat['mean'] + 2*incicator_stat['std'],incicator_stat['mean'] - 2*incicator_stat['std']

(1.6364939882897183, -1.3833052309693343)

# signal check

In [11]:
import plotly.offline as pyoff
import plotly.graph_objs as go
 
x_values = df_signal.reset_index()['index'].apply(str)
# x_values = [i for i in range(len(df_signal))]
trace1 = go.Scatter(
    x=x_values,
    y=df_signal['price'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='price',
    xaxis='x', 
    yaxis='y2'#标明设置一个不同于trace1的一个坐标轴
)

trace2 = go.Scatter(
    x=x_values,
    y=df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA200'
)

trace3 = go.Scatter(
    x=df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA100'
)

trace4 = go.Scatter(
    x=df_signal['log-OFI-MA50'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['log-OFI-MA50'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA50'
)
 
data = [trace1,trace2,trace3,trace4]
data = [trace1,trace2]
layout = go.Layout({"template": 'simple_white',
                   "title":{"text": 'price & log-OFI-MA', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})
 
fig = go.Figure(data=data, layout=layout)
fig.show()


In [12]:
from statsmodels.tsa.stattools import adfuller
if adfuller(df_signal['log-OFI-MA50'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].dropna())[1] < 0.001:
    print("This series is stationary!")
else:
    print("This series is not stationary!")

This series is stationary!


In [13]:
# assert False

In [14]:
import plotly.graph_objects as go
 
fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=df_signal.index, y=df_signal['log-OFI']),
    
],
)

fig.show()    

In [15]:
# df_stk = df_stk_orgin.copy(deep=True)
# df_stk['date_time'] = df_stk['date_time'].apply(lambda x:pd.to_datetime(str(x)))
# df_stk = df_stk.set_index('date_time')

In [16]:
df_signal = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']
# df_stk = df_stk.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']

In [17]:
# df_position = pd.DataFrame(data=[],columns=['pos'])
# # sigma = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['std']
# # mean = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['mean']
# # short_threshold = mean + 2*sigma
# # long_threshold = mean - 2*sigma
# short_threshold = 10
# long_threshold = -10
# for index,row in df_signal.iterrows():
#     indicator = row['log-OFI-MA100']
#     if row['log-OFI'] > short_threshold:
#         df_position.loc[index,'pos'] = 1
#     elif row['log-OFI'] < long_threshold:
#         df_position.loc[index,'pos'] = -1
#     else:
#         df_position.loc[index,'pos'] = 0
#     df_position.loc[index,'price'] = row['price']

In [18]:
df_position = pd.DataFrame(data=[],columns=['pos'])

short_open_threshold = 0.973773 #做空开仓
short_close_threshold = 0 #做空平仓

long_close_threshold = -0 #做多平仓
long_open_threshold = -1.407495 #做多开仓
 
intraday_open_times = 0
max_open = 3

long_open_flag = 0
short_open_flag = 0

for index,row in df_signal.iterrows():
    indicator = row['log-OFI-MA200']
    
    if indicator >= short_open_threshold and short_open_flag==0 and long_open_flag==0:
        if intraday_open_times < max_open:
            #diff落在A区间，且无仓位，做空
            short_open_flag = 1
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
            intraday_open_times += 1
        else:
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
    elif indicator >= short_open_threshold and short_open_flag==0 and long_open_flag==1:
        if intraday_open_times < max_open:
            #diff落在A区间，且有多头仓位，平仓再做空
            short_open_flag = 1
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
            intraday_open_times += 1
        else:
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
    elif indicator >= short_open_threshold and short_open_flag==1 and long_open_flag==0:
        #diff落在A区间，且有空头仓位，keep
        short_open_flag = 1
        long_open_flag = 0
        df_position.loc[index,'pos'] = -1*short_open_flag


    if indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==1 and long_open_flag==0:
        #diff落在B区间，且有空头仓位，keep
        short_open_flag = 1
        long_open_flag = 0
        df_position.loc[index,'pos'] = -1*short_open_flag
    elif indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==0 and long_open_flag==1:
        #diff落在B区间，且有多头仓位，平仓
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = 1*long_open_flag
    elif indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==0 and long_open_flag==0:
        #diff落在B区间，且无仓位，keep
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = -1*short_open_flag


    if indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==1 and long_open_flag==0:
        #diff落在C区间，且有空头仓位，平仓
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = -1*short_open_flag
    elif indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==0 and long_open_flag==1:
        #diff落在C区间，且有多头仓位，平仓
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = 1*long_open_flag
    elif indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==0 and long_open_flag==0:        
        #diff落在C区间，且无仓位，keep
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = 1*long_open_flag

    if indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==1 and long_open_flag==0:
        #diff落在D区间，且有空头仓位，平仓
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = -1*short_open_flag
    elif indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==0 and long_open_flag==1:
        #diff落在D区间，且有多头仓位，keep
        short_open_flag = 0
        long_open_flag = 1
        df_position.loc[index,'pos'] = 1*long_open_flag
    elif indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==0 and long_open_flag==0:
        #diff落在D区间，且无仓位，keep
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = 1*long_open_flag

    if indicator <= long_open_threshold and short_open_flag==0 and long_open_flag==0:
        if intraday_open_times < max_open:
            #diff落在E区间，且无仓位，做多
            short_open_flag = 0
            long_open_flag = 1
            df_position.loc[index,'pos'] = 1*long_open_flag
            intraday_open_times += 1
        else:
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag
    elif indicator <= long_open_threshold and short_open_flag==1 and long_open_flag==0:
        if intraday_open_times < max_open:
            #diff落在E区间，且有空头仓位，平仓再做多
            short_open_flag = 0
            long_open_flag = 1
            df_position.loc[index,'pos'] = 1*long_open_flag
            intraday_open_times += 1
        else:
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag
    elif indicator <= long_open_threshold and short_open_flag==0 and long_open_flag==1:
        #diff落在E区间，且有多头仓位，keep
            short_open_flag = 0
            long_open_flag = 1
            df_position.loc[index,'pos'] = 1*long_open_flag
    
    if index.time() > time(14,50,0):
        #每天14:50之后，强制平仓
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = 1*long_open_flag


df_position = pd.concat([df_position,df_signal],axis=1)
df_position.head()

,pos,wt,vt,price,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA200
2022-08-22 09:30:00,NaN,10.199882,0.0,34.15,10.199882,NaN,NaN,NaN
2022-08-22 09:30:03,NaN,10.422876,-8.961879,34.22,19.384755,NaN,NaN,NaN
2022-08-22 09:30:06,NaN,-10.422876,9.296518,34.19,-19.719394,NaN,NaN,NaN
2022-08-22 09:30:09,NaN,10.240674,9.425452,34.01,0.815222,NaN,NaN,NaN
2022-08-22 09:30:12,NaN,-10.240674,9.035987,34.12,-19.276661,NaN,NaN,NaN


In [19]:
# assert False

# backtest

In [27]:
df_signal_bt = df_position.iloc[0:1,:].append(df_position.dropna())
df_signal_bt.iloc[0,0] = 0 
df_signal_bt.head()

,pos,wt,vt,price,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA200
2022-08-22 09:30:00,0,10.199882,0.0,34.15,10.199882,NaN,NaN,NaN
2022-08-22 09:38:41,0,8.388678,-0.004283,33.80,8.39296,0.197392,-1.373037,0.111691
2022-08-22 09:38:44,0,0.066002,10.047588,33.80,-9.981586,0.372538,-1.466581,0.061783
2022-08-22 09:38:47,0,-8.454679,10.250723,33.79,-18.705402,-0.376006,-1.461481,-0.036622
2022-08-22 09:38:50,0,-0.036701,-0.06949,33.79,0.032789,-0.740542,-1.462043,-0.036728


In [21]:
df_trade = pd.DataFrame([])

cash = 0 #初始账户现金
stk_hold = 0 #初始股票持有量

commission_rate = 2/10000
tax_rate = 1/1000

stk_volume = 1/3

for i in range(len(df_signal_bt)-1):

    # df_position的column
    # ['pos','wt','vt','price','log-OFI','log-OFI-MA50','log-OFI-MA100','log-OFI-MA200']
    if df_signal_bt.values[i][0] != df_signal_bt.values[i+1][0]:
        indicator = df_signal_bt.values[i+1][-1]
        print('record!',df_signal_bt.index[i+1],indicator)
        index = df_signal_bt.index[i+1]
        
        pre_pos = df_signal_bt.values[i][0]
        pos = df_signal_bt.values[i+1][0]
        
        stk_price = df_signal_bt.values[i+1][3]
        commission = commission_rate * stk_price * stk_volume
        if pre_pos == 1:
            tax = tax_rate * stk_price * stk_volume
        else:
            tax = 0

        if pos == 0 and pre_pos == 1:
            cash = cash + stk_price*stk_volume - commission - tax
            stk_hold = 0
            asset_value = 0
        elif pos == 0 and pre_pos == -1:
            cash = cash - stk_price*stk_volume - commission - tax
            stk_hold = 0
            asset_value = 0
        
        elif pos == 1 and pre_pos == 0:
            cash = cash - stk_price*stk_volume - commission - tax
            stk_hold = 1*stk_volume
            asset_value = stk_hold*stk_price
        elif pos == 1 and pre_pos == -1 :
            cash = cash + (stk_price*stk_volume - commission - tax)*2
            stk_hold = 1*stk_volume
            asset_value = stk_hold*stk_price
        
        elif pos == -1 and pre_pos == 0:
            cash = cash + stk_price*stk_volume - commission - tax
            stk_hold = -1*stk_volume
            asset_value = stk_hold*stk_price
        elif pos == -1 and pre_pos == 1:
            cash = cash + (stk_price*stk_volume - commission - tax)*2
            stk_hold = -1*stk_volume
            asset_value = stk_hold*stk_price
        
            
    ########################################################################################################################
    # asset_value = stk_hold*stk_price
        df_trade.loc[index,'log-OFI-MA200'] = np.round(indicator,4)
        df_trade.loc[index,'pos'] = pos
        df_trade.loc[index,'stock hold'] = stk_hold
        df_trade.loc[index,'price'] = stk_price
        df_trade.loc[index,'asset value'] = asset_value
        df_trade.loc[index,'cash'] = cash
        df_trade.loc[index,'commission'] = commission
        df_trade.loc[index,'tax'] = tax
    # df_trade.loc[index,'net value'] = cash + asset_value
df_trade['net value'] = df_trade['asset value'] + df_trade['cash']
# df_trade['long/short threshold'] = [str(long_threshold)+'@'+str(short_threshold)]*len(df_trade)

record! 2022-08-22 11:05:51 -1.421448028365941
record! 2022-08-22 11:18:19 0.030073840738151558
record! 2022-08-22 11:24:20 0.9953609710037454
record! 2022-08-22 13:12:06 -0.011787755352246654
record! 2022-08-22 13:38:51 0.9983726824606027
record! 2022-08-22 13:43:59 -0.007211085937694781


In [28]:
df_position

,pos,wt,vt,price,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA200
2022-08-22 09:30:00,NaN,10.199882,0.0,34.15,10.199882,NaN,NaN,NaN
2022-08-22 09:30:03,NaN,10.422876,-8.961879,34.22,19.384755,NaN,NaN,NaN
2022-08-22 09:30:06,NaN,-10.422876,9.296518,34.19,-19.719394,NaN,NaN,NaN
2022-08-22 09:30:09,NaN,10.240674,9.425452,34.01,0.815222,NaN,NaN,NaN
2022-08-22 09:30:12,NaN,-10.240674,9.035987,34.12,-19.276661,NaN,NaN,NaN
2022-08-22 09:30:14,NaN,10.3552,8.630522,34.12,1.724678,NaN,NaN,NaN
2022-08-22 09:30:17,NaN,-10.3552,-8.630522,34.13,-1.724678,NaN,NaN,NaN
2022-08-22 09:30:20,NaN,10.377234,-8.630522,34.13,19.007756,NaN,NaN,NaN
2022-08-22 09:30:23,NaN,-10.377234,9.609116,34.10,-19.986351,NaN,NaN,NaN
2022-08-22 09:30:26,NaN,-10.292823,9.268609,34.10,-19.561433,NaN,NaN,NaN


In [22]:
df_trade

,log-OFI-MA200,pos,stock hold,price,asset value,cash,commission,tax,net value
2022-08-22 11:05:51,-1.4214,1.0,0.333333,33.18,11.06,-11.062212,0.002212,0.000000,-0.002212
2022-08-22 11:18:19,0.0301,0.0,0.000000,33.04,0.00,-0.062095,0.002203,0.011013,-0.062095
2022-08-22 11:24:20,0.9954,-1.0,-0.333333,33.21,-11.07,11.005691,0.002214,0.000000,-0.064309
2022-08-22 13:12:06,-0.0118,0.0,0.000000,33.23,0.00,-0.073191,0.002215,0.000000,-0.073191
2022-08-22 13:38:51,0.9984,-1.0,-0.333333,33.30,-11.10,11.024589,0.002220,0.000000,-0.075411
2022-08-22 13:43:59,-0.0072,0.0,0.000000,33.14,0.00,-0.024287,0.002209,0.000000,-0.024287


In [23]:
df_trade_long = df_trade[df_trade['pos']==1]
df_trade_short = df_trade[df_trade['pos']==-1]
df_trade_close = df_trade[df_trade['pos']==0]

# performance analysis

In [24]:
x_values = df_trade['net value'].index
# x_values = [i for i in range(len(df_trade))]

fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=x_values, y=df_trade['net value']),
],
)
fig.show() 

In [25]:
'''
rgba颜色对照表
https://wenku.baidu.com/view/41c91c5b24284b73f242336c1eb91a37f0113257.html
plotly绘制散点图
https://www.yisu.com/zixun/168081.html
'''

x_values = df_signal.reset_index()['index'].apply(str)
# x_values = [i for i in range(len(df_signal))]
trace1 = go.Scatter(
    x=x_values,
    y=df_signal['price'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='price',
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    mode = 'lines'
)

trace2 = go.Scatter(
    x=x_values,
    y=df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA200',
    mode = 'lines'
)


trace_long = go.Scatter(
    x = df_trade_long.index,
    y = df_trade_long['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'long open',
    mode = 'markers',
    marker = dict(
    size = 8, # 设置点的宽度
    color = 'rgba(227, 23, 13, .9)'), #红色
    )

trace_short = go.Scatter(
    x = df_trade_short.index,
    y = df_trade_short['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'short open',
    mode = 'markers',
    marker = dict(
    size =8, # 设置点的宽度
    color = 'rgba(151, 51, 250, .9)'), #紫色
    )

trace_close = go.Scatter(
    x = df_trade_close.index,
    y = df_trade_close['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'close',
    mode = 'markers',
    marker = dict(
    size = 8, # 设置点的宽度
    color = 'rgba(0, 255, 0, .9)'), #绿色
    )

data = [trace1,trace2,trace_long,trace_short,trace_close]
layout = go.Layout({"template": 'simple_white',
                   "title":{"text": 'price & log-OFI-MA', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})
 
fig = go.Figure(data=data, layout=layout)
fig.show()
